# Introduction to Partial Differential Equations
---

## Chapter 3: Parabolic PDEs, the Heat Equation, and a Deep Dive into Fourier Series 
---

## Want to use Colab? [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec2.ipynb)

---

## Prepping the environment for interactive plots in Colab
---

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab - installing missing packages')
    !pip install ipympl
    from IPython.display import clear_output
    clear_output()
    exit()
else:
    print('Not running on CoLab - assuming environment has necessary packages')

In [ ]:
%matplotlib widget
if 'google.colab' in str(get_ipython()):
    from google.colab import output
    output.enable_custom_widget_manager()

## Creative Commons License Information
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Introduction to Partial Differential Equations: Theory and Computations</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" property="cc:attributionName" rel="cc:attributionURL">Troy Butler</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" rel="dct:source">https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations</a>.


***This particular notebook is motivated in large part by the Chapter 3 presentation in the OER textbook [Partial Differential Equations by Victor Ivrii](http://www.math.toronto.edu/ivrii/PDE-textbook/PDE-textbook.pdf) released under a Creative Commons Attribution-ShareAlike 4.0 International License.***

In [ ]:
import numpy as np
%matplotlib widget 
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

---
## Section 3.2: The Heat Equation on a Bounded Interval and Formal Solutions
---

Consider the initial boundary value problem (IBVP) used to model how the initial temperature in a rod of length $\ell$ evolves over time if there is no heat source over the length of the rod and the ends are kept at an ambient temperature (denoted by $0$ for simplicity), 

$$
\left\{\begin{array}{rl}
    u_t = c u_{xx}, & 0<x<\ell, t>0, \\
    u(0,t)=u(\ell,t)=0, & t>0, \\
    u(x,0) = f(x), & 0<x<\ell.
\end{array}\right.
$$

Here, $c>0$ is a thermal diffusivity parameter defining the "ease" by which heat is diffused throughout the rod. The function $f(x)$ denotes the initial temperature distribution.

In this notebook, we analyze:

- Properties of solutions to this IBVP

- Formal solution techniques to this IBVP and its variants (when the BCs are altered)

---
### Section 3.2.1: Decreasing Energy Over Time
---

Assume that $u(x,t)$ solves the above IBVP and that this function and its derivatives $u_t(x,t)$ and $u_{xx}(x,t)$ all belong to the class of functions defined by $\mathcal{C}([0,\ell]\times[0,\infty))$, i.e., they are continuous for $x\in[0,\ell]$ and $t\in[0,\infty)$.

Define the energy $E(t)$ for $t\geq 0$ as

$$
    E(t):=\int_0^\ell u^2(x,t)\, dx.
$$

Notice that $E$ is a scalar function of time $t$. We differentiate with respect to $t$ to give, for $t>0$, 

$$
   \begin{align}
       E'(t) &= \frac{d}{dt}\int_0^\ell u^2(x, t)\, dx \\
             \\
             &= \int_0^\ell \frac{\partial}{\partial t} u^2(x,t)\, dx \qquad \text{ (smooth $u$ allows us to interchange limits)} \\
             \\
             &= 2\int_0^\ell u(x,t)u_t(x,t)\, dx \qquad \text{ (chain rule)} \\
             \\
             &= 2c \int_0^\ell u(x,t)u_{xx}\, dx \qquad \text{ ($u_t=cu_{xx}$)} \\
             \\
             &= 2c\underbrace{\left[u(x,t)u_x(x,t)\right]_0^\ell}_{=0 \text{ because of BCs}} - 2c\int_0^\ell (u_x(x,t))^2 \, dx \qquad \text{ (integration by parts)} \\
             \\
             &= - 2c\int_0^\ell (u_x(x,t))^2 \, dx\\
             \\
             &\leq 0.
   \end{align}
$$

This implies that $E(t)$ is a (not necessarily strictly) decreasing function so that $E(t)\leq E(0)$. This immediately implies that for all $t\geq 0$, 

$$
    \int_0^\ell u^2(x,t)\, dx \leq \int_0^\ell f^2(x)\, dx.
$$

In [ ]:
from scipy.integrate import quad

In [ ]:
# Once we figure out what u is, we can use this function to compute 
# the energy of u as a function of time
def energy(u, ell, t):
    return quad(lambda x, t: u(x,t)**2, 0, ell, args=(t,))[0]

---
#### Student Activity
---

Use the above property that energy is decreasing to prove that

- the IBVP is stable with respect to perturbations in the IC, 

- and that solutions, if they exist, are unique.

(We will discuss the actual form of solutions below.)

<mark>Student proof of stability goes below.</mark>

<mark>Student proof of uniqueness of solutions goes below.</mark>

### Section 3.2.2: A 3-Step Process to Formally Solve the Heat Equation
---
We formally solve this IBVP using a 3-step process. <mark>***Students should review the similar approach taken in [Section 2.6](../Chp2/Chp2Sec6.ipynb) to solve the 2D Poisson equation on a rectangular domain.***</mark>

1. **Apply separation of variables**. 

   We first ignore the IC $u(x,0)=f(x)$ and find a family of functions $\{u_k(x,t)\}_{k=1}^\infty$ of the form $\{X_k(x)T_k(t)\}_{k=1}^\infty$ satisfying the differential equation and BCs. 
   
   Writing a potential solution as $X(x)T(t)$ is referred to as a separation of variables. 

2. **Apply the principle of superposition**. 

   We demonstrate (not rigorously as that will come later) that for any "appropriate" sequence of real numbers $\{c_k\}_{k=1}^\infty\subset\mathbb{R}$ that $u(x,t) = \sum_{k=1}^\infty c_k u_k(x,t)$ also satisfies the differential equation and BCs. 

3. **Use the Fourier coefficients of $f(x)$ to determine the coefficients $\{c_k\}_{k=1}^\infty$**. 

   Write $f(x)=\sum_{k=1}^\infty c_k u_k(x,0)$ and exploit orthogonality of the $X_k$ functions to determine $c_k$ for each $k\in\mathbb{N}$. 

This is a ***formal*** process because there is plenty of justification missing. We will fill in the much of the missing justification in [Section 3.3](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec3.ipynb).

---
### Section 3.2.3: Separation of variables</a>
---

We ignore the IC and make the [ansatz](https://en.wikipedia.org/wiki/Ansatz) that a function of the form

$$
    u(x,t) = X(x)T(t)
$$

satisfies the differential equation and BCs. *By comparison, in [Section 2.6](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2/Chp2Sec6.ipynb), we assumed solutions took the form $X(x)Y(y)$ for the 2D Poisson equation on the rectangle.*

Inserting this ansatz into the differential equation gives

$$
    X(x)T'(t) = cX''(x)T(t).   
$$

Since we are interested in non-trivial solutions, we are not interested in cases where $u(x,t)=X(x)T(t)$ is identically zero, which also means that we are not interested in cases where either $X(x)$ or $T(t)$ are identically zero. Thus, we divide through by $X(x)$ and $T(t)$ to get

$$
    \frac{T'(t)}{T(t)} = c\frac{X''(x)}{X(x)}.
$$

The left-hand side depends only on $t$ whereas the right-hand side depends only on $x$. Thus, if we kept $t$ fixed and varied $x$, we see that $\frac{X''(x)}{X(x)}$ is constant and vice versa. It follows that both expressions are equal to the same constant, which (with some mathematical foresight) we denote by $-c\lambda$ so that

$$
    \frac{T'(t)}{T(t)} = c\frac{X''(x)}{X(x)} = -c\lambda.
$$

This defines the following two ODEs:

$$
\begin{align}
    X''(x) + \lambda X(x) &= 0, \\ \\
    T'(t) +c\lambda T(t)  &= 0.
\end{align}
$$

Since we are assuming the BCs are satisfied, this means that $X(0)=X(\ell)=0$. By a straightforward modification of the domain of the problem used to construct eigenfunctions derived in [Section 2.5](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2/Chp2Sec5.ipynb), we have that for any $k\in\mathbb{N}$ that the pair $(X_k, \lambda_k)$ defined by

$$
    \lambda_k = \frac{(k\pi)^2}{\ell^2}
$$

and

$$
    X_k(x) = \sin\left(\frac{k\pi x}{\ell}\right)
$$

satisfies $X_k''(x)+\lambda_k X_k(x) = 0$ and the BCs.

For such a $\lambda_k=\frac{(k\pi)^2}{\ell^2}$, we have that 

$$
    T_k(t) = T_k(0)\exp\left(-c\frac{(k\pi)^2}{\ell^2} t\right)
$$

satisfies $T_k'(t)+c\lambda_k T_k(t)$ where $T_k(0)$ is some unspecified IC. Since we are not initially concerned with any initial condition in particular, we might as well choose $T_k(0)=1$ for simplicity so that

$$
    T_k(t) = \exp\left(-c\frac{(k\pi)^2}{\ell^2} t\right). 
$$

This implies that for each $k\in\mathbb{N}$, 

$$
    u_k(x,t) = \exp\left(-c\frac{(k\pi)^2}{\ell^2} t\right) \sin\left(\frac{k\pi x}{\ell}\right)
$$

satisfies the differential equation and BCs.

**Remarks:**

- For each $k$, the choice of $T_k(0)=1$ implies $u_k(x,0)=X_k(x)=\sin\left(\frac{k\pi x}{\ell}\right)$. Thus, while each $u_k$ satisfies the same differential equation and BCs, they all satisfy different ICs. We return to this below.

   - The $u_k$ above are sometimes called *particular solutions*, which can be confusing since they have nothing to do with how this term is used in ODEs.


- It is clear that $c>0$ implies that for each $x\in(0,\ell)$ that $u_k(x,t)\to 0$ as $t\to\infty$ and that larger $c$ imply a faster rate of decay. In other words, a larger thermal diffusivity implies the temperature changes faster (no surprise there).

- If the rod is made longer (so $\ell$ increases) this makes $c/\ell$ smaller. It takes longer for the temperature to decay to zero for a longer rod. 

  - Does this make sense? Suppose $\ell_1>\ell_2$ denotes the lengths of two rods of the same material. If we consider the midpoint of each rod, the midpoint of the first rod is *farther* away from the endpoints where temperature is kept at $0$. Think of the endpoint values as a forcing term at the ends of the rods. It makes sense that it takes longer for this force to be "felt" at the midpoint of the first, longer rod compared to the second, shorter rod. 

In [ ]:
def X_k(x, ell, k):
    return np.sin(np.pi*k*x/ell)

def T_k(t, c, ell, k):
    return np.exp(-c*(k*np.pi/ell)**2*t)

def u_k(x, t, c, ell, k):
    return T_k(t, c, ell, k)*X_k(x, ell, k)

In [ ]:
def plot_u_k(t, c, ell, k):
    
    x = np.linspace(0, ell, 200)
    
    plt.figure(0, figsize=(4,2))
    plt.clf()
    plt.plot(x, u_k(x, t, c, ell, k))
    plt.axhline(c='k', ls=':')
    plt.ylim([-1, 1])
    plt.show()

In [ ]:
%reset -f out 

%matplotlib widget
interact(plot_u_k, 
            t = widgets.FloatSlider(value=0, min=0, max=5, step=0.1, continuous_update=False),
            c = widgets.FloatSlider(value=0.1, min=0.01, max=1, step=0.01, continuous_update=False),
            ell = widgets.FloatSlider(value=1, min=0.5, max=2, step=0.1, continuous_update=False),
            k = widgets.IntSlider(value=1, min=1, max=10, step=1, continuous_update=False)
        )

In [ ]:
ell = 10  # Rod of length ell
c = 1
k = 3

u = lambda x, t: u_k(x, t, c, ell, k)

In [ ]:
# Let's plot the energy of these particular solutions

num_t = 100

t = np.linspace(0, 5, num_t)

E = np.zeros(num_t)
for i in range(num_t):
    E[i] = energy(u, ell, t[i])

%matplotlib widget
plt.figure(1)
plt.plot(t, E)
plt.title('$E(t)$ vs. $t$')
plt.show()

---
### Section 3.2.4: Principle of Superposition</a>
---


As discussed above, for a given $k\in\mathbb{N}$, the function

$$
    u_k(x,t) = \exp\left(-c\frac{(k\pi)^2}{\ell^2} t\right) \sin\left(\frac{k\pi x}{\ell}\right)
$$

satisfies the IBVP if $f(x)$ is chosen to be $\sin\left(\frac{k\pi x}{\ell}\right)$.

It is straightforward to show that if there exists constants $N$ constants, denoted by $c_1, \ldots, c_N$, such that

$$
    f(x) = \sum_{k=1}^N c_k \sin\left(\frac{k\pi x}{\ell}\right),
$$

then

$$
    u(x,t) = \sum_{k=1}^N c_k u_k(x,t)
$$

satisfies the IBVP with this choice of $f(x)$.

----
#### Two wishes
---

We have but two simple wishes:

1. Understand what it means if $N=\infty$ in the above summations.

2. Given a more general function $f(x)$, figure out how to determine the associated $c_k$ values even if it means that $N$ may need to be infinite.

While we address the second wish in Section 3.2.5 below, we only provide some hint as to the first wish in the motivation below. We will address this first wish in more detail in [Section 3.3](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec3.ipynb).

***Some motivation for these wishes.***

The reason we wish to understand what is meant if $N$ is infinite is because even simple choices of $f(x)$ may be impossible to represent as finite linear combinations of sine functions. For instance, consider $f(x)\equiv 1$ on a rod of unit length. 

However, if we allow infinite linear combinations, then as we will see, we have that

$$
    \large 1 = \frac{4}{\pi} \sum_{k=1}^\infty \frac{1}{2k-1}\sin((2k-1)\pi x).
$$

Let $S_N(x)$ denote a partial sum where the first $N$ terms are taken. 

We make some plots below to show that $S_N(x)$ does converge, in a sense, to $1$. However, it is more complicated as it initially appears (we will see this by taking the discretization of $[0,1]$ to be very fine to demonstrate that there is fact no pointwise convergence at the endpoints as is suggested when the discretization is "coarse" relative to the size of $N$ used). 

In [ ]:
Ns = np.array([3, 10, 100]) 

# The use of 2*Ns.max() below ensures that we resolve oscillations
# that are ALWAYS present as N increases. 
# Try x=np.linspace(0, 1, 100) instead and observe how the converges
# appears pointwise, but this is just a resolution error.
x = np.linspace(0, 1, 2*Ns.max())  

%matplotlib widget
plt.figure(1)
for N in Ns:
    S_N = 0*x
    for k in range(1,N+1):
        S_N += 1./(2*k-1)*np.sin((2*k-1)*np.pi*x)
    S_N *= 4/np.pi
    plt.plot(x,S_N, label='N='+str(N))

plt.legend(loc='lower center', shadow=True)

---
#### Motivating a different notion of converence
---

It appears that the error in the $\sup$-norm is about $0.2$ on $(0,1)$ for just about every function in the partial sum! 
In other words, it seems like if we let $f(x)=1$ and $S_N(x)$ be defined as above, then it appears (at least in the eye-ball norm) that 

$$
    \large \sup_{x\in[0,1]} | f(x) - S_N(x) | \approx 0.2
$$

seems to be true for an infinite number of $N$ values. (In fact, the actual distance is $1$ because of the discrepancy at the endpoints.)

We recall from basic calculus/analysis that for a sequence of real numbers $(a_n)$ and real number $a$ that if there exists an $\epsilon>0$ such that no matter what $N$ is chosen there is always an $n>N$ such that $|a_n-a|\geq \epsilon$, then $(a_n)$ does **not** converge to $a$. 

We clearly need a different notion of convergence if we are to understand what is meant by $\lim_{N\to\infty} S_N(x)$.

---
#### A different notion of convergence
---

Suppose that instead of defining the error in the partial sum approximation as $e_N = \sup_{x\in[0,1]} |f(x)-S_N(x)|$, we decided to measure the <mark>***mean square error (MSE)***</mark>, i.e., we define the square of the error in an ***average*** sense as

$$
    \large e_N := \int_0^1 |f(x)-S_N(x)|^2\, dx.
$$

If this $e_N\to 0$ as $N\to\infty$, then we say that the sequence of partial sums converges in mean square (or we say that it converges in $L^2$ if we have taken measure theory/functional analysis). 

---
#### Approximating the integrals defining the MSE
---

Since the integrand involved in computing $e_N$ looks annoying/irritating to compute, we use [Monte Carlo (MC)](https://en.wikipedia.org/wiki/Monte_Carlo_integration) approximations because they are incredibly simple to implement.

MC approximations will generally vary from sample set to sample set, but the variance can be made sufficiently small if a sufficiently large number of samples are used thanks to the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem). 

In [ ]:
M = 1E6  # Number of MC sample points

x = np.random.random(int(M))

for N in Ns:
    S_N = 0*x
    for k in range(1,N+1):
        S_N += 1./(2*k-1)*np.sin((2*k-1)*np.pi*x)
    S_N *= 4/np.pi
    e_N = 1/M * np.sum((1-S_N)**2)
    print(e_N)

---
#### The smoothing effect of the heat equation
---

Below, we show the solution (or at least a truncated version) of the IBVP at various times with this initial condition of $f(x)\equiv 1$.

***Note that within $t=1E-5$ that the oscillations are already removed.***

The heat equation *smooths* rough data very quickly (something we already knew from [Section 3.1](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec1.ipynb)). This is good news for us considering that $f(x)\equiv 1$ is not rough but our approximations to it are.

Why do the "rough" parts of our truncation get smoothed out so quickly? Well, they are due to the "large" $k$ terms showing up in the summation, and a quick glance at the time component of the solution shows that larger values of $k$ (which correspond to the "faster oscillation" parts in space) are damped in time very quickly.

In [ ]:
N = 100  # Specify the truncation
ts = [0.00001, 0.01, 0.1]  # Specify the times to plot the solution
x = np.linspace(0, 1, 2*N)

%matplotlib widget
plt.figure(2)
for t in ts:
    u_N = 0*x
    for k in range(1,N+1):
        u_N += 1./(2*k-1)*np.sin((2*k-1)*np.pi*x) * \
                    np.exp(-((2*k-1)*np.pi)**2*t)
    u_N *= 4/np.pi
    plt.plot(x,u_N, label='t='+str(t))

plt.legend(loc='lower center', shadow=True)

---
### Section 3.2.5: Fourier Coefficients
---

We follow the process that was first described in [Section 2.5](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2/Chp2Sec5.ipynb) with a few adjustments.

First, we adjust the continuous inner product, denoted by $\langle \cdot, \cdot \rangle$, to be defined as an integral over $[0,\ell]$.

We then recognize that for $k, m\in\mathbb{N}$, 

$$
    \langle X_k(x), X_m(x)\rangle = \left\langle \sin\left(\frac{k\pi x}{\ell}\right), \sin\left(\frac{m\pi x}{\ell}\right) \right\rangle 
                                  =\begin{cases}
                                        0, & k\neq m, \\
                                        \ell/2, & k=m.
                                    \end{cases}
$$

It follows that if

$$
    f(x) = \sum_{k=1}^\infty c_k\sin\left(\frac{k\pi x}{\ell}\right), 
$$

then for each $k\in\mathbb{N}$, 

$$
    c_k := \frac{2}{\ell}\left\langle f(x), \sin\left(\frac{k\pi x}{\ell}\right)\right\rangle = \frac{2}{\ell}\int_0^\ell f(x) \sin\left(\frac{k\pi x}{\ell}\right)\, dx.
$$

The coefficients $c_k$ are referred to as **Fourier coefficients** and the corresponding series is called a **Fourier series** (although in this case it is more precise to say **Fourier sine series**).

We then set the formal solution to the IBVP as

$$
    u(x,t) = \sum_{k=1}^\infty c_k  \exp\left(-c\frac{(k\pi)^2}{\ell^2} t\right) \sin\left(\frac{k\pi x}{\ell}\right)
$$

with the $c_k$ values as defined above.

---
#### Student Activity
---

*This activity goes into the details of a result studied above.* 

Show that if $f(x)=1$ and $\ell=1$, then

$$
    c_k = \begin{cases}
            \frac{4}{k\pi}, & k=1, 3, 5,\ldots, \\
            0, & k=2, 4, 6, \ldots, 
          \end{cases}
$$

so that

$$
    1 = \frac{4}{\pi} \sum_{k=1}^\infty \frac{1}{2k-1}\sin((2k-1)\pi x).
$$

Plot some partial Fourier sums for this function as well as an approximation to the solution of the IBVP at $t=0.001, 0.01$, and $0.1$ by truncating the series at $N=10, 50$ and $100$.

**Activity Solution - Unhide the cells below to see the solution (students should attempt this on their own before referring to the solutions to check their work)**

Since $f(x)=1$ and $\ell=1$, for any $n\in\mathbb{N}$ (the use of $n$ as an index provides a more straightforward change of index to $k$ later), 

$$
\begin{align}
    c_n &= 2\int_0^1 \sin(n\pi x)\, dx \\ \\
        &= -\frac{2}{n\pi} \big[\cos(n\pi x)\big]_0^1 \\ \\ 
        &= -\frac{2}{n\pi} \left[\cos(n\pi) - \cos(0)\right] \\ \\
        &= -\frac{2}{n\pi} \left[\cos(n\pi) - 1 \right]
\end{align}
$$

If $n$ is even, then $n=2k$ for some $k\in\mathbb{N}$, and $\cos(2k\pi)=1$. This implies that $c_{2k}=0$. 

Otherwise, $n$ is odd, so $n=2k-1$ for some $k\in\mathbb{N}$, and $\cos((2k-1)\pi)=-1$. This implies that

$$
    c_{2k-1} = -\frac{2}{(2k-1)\pi}[-2] = \frac{4}{\pi}\frac{1}{2k-1}.
$$

Thus, 

$$
    1 = \frac{4}{\pi} \sum_{k=1}^\infty \frac{1}{2k-1}\sin((2k-1)\pi x).
$$

In [ ]:
# We now plot some partial Fourier sums for this function

Ns = np.array([10, 50, 100]) 

# The use of 2*Ns.max() below ensures that we resolve oscillations
# that are ALWAYS present as N increases. 
# Try x=np.linspace(0, 1, 100) instead and observe how the converges
# appears pointwise, but this is just a resolution error.
x = np.linspace(0, 1, 2*Ns.max())  

%matplotlib widget
plt.figure(1)
for N in Ns:
    S_N = 0*x
    for k in range(1,N+1):
        S_N += 1/(2*k-1)*np.sin((2*k-1)*np.pi*x)
    S_N *= 4/np.pi
    plt.plot(x,S_N, label='N='+str(N))

plt.legend(loc='lower center', shadow=True)

In [ ]:
# We now plot the solution to the IBVP using N=10 terms to approximate the IC

N = 10  # Specify the truncation
ts = [0.001, 0.01, 0.1]  # Specify the times to plot the solution
x = np.linspace(0, 1, 200)

%matplotlib widget
plt.figure(2)
for t in ts:
    u_N = 0*x
    for k in range(1,N+1):
        u_N += 1./(2*k-1)*np.sin((2*k-1)*np.pi*x) * \
                    np.exp(-((2*k-1)*np.pi)**2*t)
    u_N *= 4/np.pi
    plt.plot(x,u_N, label='t='+str(t))

plt.legend(loc='lower center', shadow=True)

In [ ]:
# We now plot the solution to the IBVP using N=50 terms to approximate the IC

N = 50  # Specify the truncation
ts = [0.001, 0.01, 0.1]  # Specify the times to plot the solution
x = np.linspace(0, 1, 200)

%matplotlib widget
plt.figure(2)
for t in ts:
    u_N = 0*x
    for k in range(1,N+1):
        u_N += 1./(2*k-1)*np.sin((2*k-1)*np.pi*x) * \
                    np.exp(-((2*k-1)*np.pi)**2*t)
    u_N *= 4/np.pi
    plt.plot(x,u_N, label='t='+str(t))

plt.legend(loc='lower center', shadow=True)

In [ ]:
# We now plot the solution to the IBVP using N=100 terms to approximate the IC

N = 100  # Specify the truncation
ts = [0.001, 0.01, 0.1]  # Specify the times to plot the solution
x = np.linspace(0, 1, 200)

%matplotlib widget
plt.figure(2)
for t in ts:
    u_N = 0*x
    for k in range(1,N+1):
        u_N += 1./(2*k-1)*np.sin((2*k-1)*np.pi*x) * \
                    np.exp(-((2*k-1)*np.pi)**2*t)
    u_N *= 4/np.pi
    plt.plot(x,u_N, label='t='+str(t))

plt.legend(loc='lower center', shadow=True)

In [ ]:
# We now plot the difference in solutions to the IBVP using N=50 and 100 terms to approximate the IC

Ns = [50, 100]  # Specify the truncations
ts = [0.00001, 0.001]  # Specify the times to plot the differences in solutions
x = np.linspace(0, 1, 200)

%matplotlib widget
plt.figure(2)
for t in ts:
    u_Ns0 = 0*x
    for k in range(1,Ns[0]):
        u_Ns0 += 1./(2*k-1)*np.sin((2*k-1)*np.pi*x) * \
                    np.exp(-((2*k-1)*np.pi)**2*t)
    u_Ns0 *= 4/np.pi
    
    u_Ns1 = 0*x
    for k in range(1,Ns[1]):
        u_Ns1 += 1./(2*k-1)*np.sin((2*k-1)*np.pi*x) * \
                    np.exp(-((2*k-1)*np.pi)**2*t)
    u_Ns1 *= 4/np.pi
    
    plt.plot(x, u_Ns0-u_Ns1, label='t='+str(t))

plt.legend(loc='lower center', shadow=True)

---
#### Student Activity
---

Show that if $f(x)=x$ and $\ell=1$, then

$$
    c_k = \frac{2}{k\pi}(-1)^{k+1}. 
$$

Plot some partial Fourier sums for this function as well as an approximation to the solution of the IBVP at $t=0.001, 0.01$, and $0.1$ by truncating the series at $N=10, 50$ and $100$.

**Activity Solution - Unhide the cells below to see the solution (students should attempt this on their own before referring to the solutions to check their work)**

Since $f(x)=x$ and $\ell=1$, for any $k\in\mathbb{N}$, 

$$
\begin{align}
    c_k &= 2\underbrace{\int_0^1 x\sin(k\pi x)\, dx}_{\text{integrate by parts}} \\ \\
        &= \frac{2}{k\pi}\left\{ \underbrace{\big[-x\cos(k\pi x)\big]_0^1}_{=-\cos(k\pi)=(-1)^{k+1}} + \underbrace{\int_0^1 \cos(k\pi x)\, dx}_{=0 \text{ b/c } \sin(k\pi)=\sin(0)=0}  \right\} \\ \\
        &= \frac{2}{k\pi} (-1)^{k+1}
\end{align}
$$

In [ ]:
# We now plot some partial Fourier sums for this function

Ns = np.array([10, 50, 100]) 

# The use of 2*Ns.max() below ensures that we resolve oscillations
# that are ALWAYS present as N increases. 
# Try x=np.linspace(0, 1, 100) instead and observe how the converges
# appears pointwise, but this is just a resolution error.
x = np.linspace(0, 1, 2*Ns.max())  

%matplotlib widget
plt.figure(1)
for N in Ns:
    S_N = 0*x
    for k in range(1,N+1):
        S_N += 2/k * (-1)**(k+1) * np.sin(k*np.pi*x)
    S_N *= 1/np.pi
    plt.plot(x,S_N, label='N='+str(N))

plt.legend(loc='lower center', shadow=True)

Students should create code cells to plot the various approximations to the solution at the specified times using different truncation values

---
### Section 3.2.6: Other Boundary Conditions
---

Here, we rewrite the IBVP as

$$
\left\{\begin{array}{rl}
    u_t = c u_{xx}, & 0<x<\ell, t>0, \\
    BCs = BVs, & t>0, \\
    u(x,0) = f(x), & 0<x<\ell.
\end{array}\right.
$$

Above, BCs represent the Boundary Conditions and BVs represent the Boundary Values applied at $x=0$ and $x=\ell$.

There are four main types of BCs and two-subtypes of BVs for three of the BC types.

- **Dirichlet.** 

  Specifies the values at the boundary. Models a way of controlling the value. For the heat equation, these are fairly unrealistic models of reality.

    - Homogeneous Dirichlet: $u(0,t)=0$ and $u(\ell,t)=0$.
    <br><br>
    
    - Inhomogeneous Dirichlet: $u(0,t)=a(t)$ and $u(\ell,t)=b(t)$.


- **Neumann.** 

  Specifies the flux at the boundary. Models some measure of insulation/control of heat flux of the boundary. A more realistic model of reality than Dirichlet. 

    - Homogeneous Neumann: $u_x(0,t)=0$ and $u_x(\ell,t)=0$.
    <br><br>
    
    - Inhomogeneous Neumann: $u_x(0,t)=a(t)$, and $u_x(\ell,t)=b(t)$.


- **Robin (pronounced "Row-ban").** 

  A weighted combination of Neumann and Dirichlet that is perhaps a more realistic model than either by themselves.

    - Homogeneous Robin: $au_x(0,t) + bu(0,t)=0$ and $au_x(\ell, t) + bu_x(\ell,t)=0$ (here $a$ and $b$ are numbers defining the linear combination of the Dirichlet and Neumann conditions).
    <br><br>
    
    - Inhomogeneous Neumann: $au_x(0,t) + bu(0,t)=\alpha(t)$ and $au_x(\ell, t) + bu_x(\ell,t)=\beta(t)$.


- **Periodic.** 

  Models situations where behavior is repeated and we have identified a length scale, $\ell$, containing some integer multiple of periods of the behavior.

    - There is only one representation of this: $u(0,t)=u(\ell,t)$ and $u_x(0,t)=u_x(\ell,t)$.

---
### Section 3.2.7: The Homogeneous Neumann Case and Cosine Functions
---

Following the same steps as above for separation of variables yields:

$$
\begin{align}
    X_k''(x) + \lambda_k X_k(x) &= 0, \\ \\
    T_k'(t) +c\lambda_k T_k(t)  &= 0.
\end{align}
$$

The difference is in the boundary conditions for the $X_k$ functions which are now $X_k'(0)=X_k'(\ell)=0$.

As we again expect from [Section 2.5](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2/Chp2Sec5.ipynb), the eigenvalues are of the form $\lambda_k=\beta_k^2$ for some nonnegative $\beta_k$ and eigenfunctions are of the form

$$
    X_k = c_1\cos(\beta_k x) + c_2\sin(\beta_k x).
$$

However, unlike the results above, the boundary conditions now give that $c_2=0$, we choose $c_1=1$ and $\beta_k=k\pi/\ell$ where $k=\mathbb{N}\cup\{0\}$ (i.e., $0$ is now an allowable eigenvalue). 

<br><br>

---
#### Lemma 3.2.1: Eigenvalues and Eigenfunctions for the Homogeneous Neumann Case

For the ODE $ X_k''(x) + \lambda_k X_k(x)=0$ with homogeneous Neumann boundary conditions, the eigenvalues are $\lambda_k=(k\pi/\ell)^2$ and the eigenfunctions are $X_k(x) = \cos(k\pi x/\ell)$ for $k=\mathbb{N}\cup\{0\}$.  

---

<br><br>

Just as we saw in Section 2.5 for the inner products of sine functions, we have the following result for inner products of cosine functions.

<br><br>

---
#### Lemma 3.2.2: Inner products of Eigenfunctions for the Homogeneous Neumann Case

$$
    \langle \cos(k\pi x/\ell), \cos(m\pi x/\ell) \rangle = \begin{cases} 
                                                               0, & k\neq m, \\
                                                               \ell/2, & k=m>0, \\
                                                               \ell, & k=m=0.
                                                           \end{cases}
$$

---

<br><br>

Now that we see that $X_k$ has the form above for a given $\lambda_k=(k\pi/\ell)^2$, we also have that

$$
    T_k(t) = T_k(0)\exp\left(-c\frac{(k\pi)^2}{\ell^2} t\right)
$$

with the only difference from before being that this is for $k\in\mathbb{N}\cup\{0\}$ instead of just $k\in\mathbb{N}$ as in the Dirichlet case shown at the top of this notebook.

Thus, functions of the form

$$
    u_k(x,t) = \exp\left(-c\frac{(k\pi)^2}{\ell^2} t\right) \cos\left(\frac{k\pi x}{\ell}\right)
$$

satisfies the differential equation and (now homogeneous Neumann) BCs.

---
### Section 3.2.8: Solving the Homogeneous Neumann Heat Equation
---

For a given IC, we write its Fourier cosine expansion as

$$
    f(x) = \frac{c_0}{2} + \sum_{k=1}^\infty c_k \cos\left(\frac{k\pi x}{\ell}\right),
$$

and the corresponding solution to the IBVP as

$$
    u(x,t) = \frac{c_0}{2}+ \sum_{k=1}^\infty c_k u_k(x,t).
$$

The game is now to determine $c_k$ for $k=0,1,2,\ldots$ for a given $f$, and it should come as almost no surprise that these are now given by

$$
    c_k := \frac{2}{\ell}\left\langle f(x), \cos\left(\frac{k\pi x}{\ell}\right)\right\rangle = \frac{2}{\ell}\int_0^\ell f(x) \cos\left(\frac{k\pi x}{\ell}\right)\, dx.
$$

---
#### Student Activity
---

- Determine the Fourier cosine expansions of $f(x)=1$ and $f(x)=x$ on $[0,\ell]$. Compare to the Fourier sine expansions of these functions previously determined in this notebook.

- Plot several partial sum approximations to these Fourier cosine expansions for various choices of $N$.

- Determine the formal solutions to the homogeneous Neumann heat equation with IC given by $f(x)=1$ and $f(x)=x$.

- Plot several of the partial sum approximations to these formal solutions for various $0<t<1$ and plot the energy of these approximations for $0\leq t\leq 1$. Comment on these plots.

---
## Navigation

- [Previous](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec1.ipynb)

- [Next](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec3.ipynb)

---